In [100]:
! pip install quantum_decomp

In [101]:
import numpy as np
from qiskit import *
from math import pi, sqrt, floor
import quantum_decomp as qdp
import csv
from qiskit.providers.ibmq.managed import IBMQJobManager

In [102]:
# Transpile for simulator
TOKEN = "73db33693a123763f988ae7c477f9de052f436c01d5b9f5023e2356d1bdb47a71bc66669e76cdade311e90a50d59b66b5afd80f998d0c567cab6f696e275eb67"
IBMQ.save_account(TOKEN, overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()
#backend = provider.get_backend('ibmq_qasm_simulator')
backend = provider.backends(filters=lambda x: x.configuration().n_qubits >= 2
                                    and not x.configuration().simulator
                                    and x.status().operational==True)[0]

ibmqfactory.load_account:WARNING:2022-01-06 14:16:30,088: Credentials are already in use. The existing account in the session will be replaced.


In [104]:
line = 0
num = 0
indices = np.zeros((4800, 3, 2), dtype=int)
states = np.zeros((4800, 3, 4), dtype=complex)
antibases = np.zeros((4800, 4, 4), dtype=complex)

with open("antibases.csv") as antibases_file:
    antibases_reader = csv.reader(antibases_file)
    for row in antibases_reader:
        if line % 8 == 0:
            for i in range(3):
                indices[int(line / 8)][i][0] = int(row[2*i])
                indices[int(line / 8)][i][1] = int(row[2*i + 1])
                num = int(line / 8)
        elif line % 8 <= 3:
            for i in range(4):
                states[num][line % 8 - 1][i] = eval(row[i].replace('I', '1j'))
        else:
            for i in range(4):
                antibases[num][line % 8 - 4][i] = eval(row[i].replace('I', '1j'))
        line+=1

In [105]:
unitary = np.zeros((4, 4), dtype=complex)
circs = []

for i in range(1):    
    for j in range(4):
        unitary[j] = antibases[i][j]
    unitary = unitary.transpose()
    for k in range(3):
        qc = QuantumCircuit(2)
        qc_gates = qdp.matrix_to_qiskit_circuit(unitary)

        # construct circuits
        qc.initialize(states[i][k], qubits=[0, 1])
        qc.barrier()
        qc += qc_gates.inverse()
        qc.measure_all()
        
        circs.append(qc)

#backend = Aer.get_backend('aer_simulator')
circs = transpile(circs, backend)
print(circs[0])

# Use Job Manager to break the circuits into multiple jobs.
runs = 1000
job_manager = IBMQJobManager()
job_set = job_manager.run(circs, backend=backend, name='jobs', shots=runs)

# Run and get counts
#result = backend.run(qc, shots=runs).result()
results = job_set.results()
counts = result.get_counts(1)
print(counts)

global phase: 0
                ░ ┌─────────┐┌───┐┌──────────┐┌───┐┌────┐┌────────────┐┌────┐»
      q_0 -> 0 ─░─┤ RZ(π/2) ├┤ X ├┤ RZ(3π/2) ├┤ X ├┤ √X ├┤ RZ(4.2487) ├┤ √X ├»
                ░ └─────────┘└─┬─┘└──────────┘└─┬─┘└────┘└────────────┘└────┘»
      q_1 -> 1 ─░──────────────■────────────────■────────────────────────────»
                ░                                                            »
ancilla_0 -> 2 ──────────────────────────────────────────────────────────────»
                                                                             »
ancilla_1 -> 3 ──────────────────────────────────────────────────────────────»
                                                                             »
ancilla_2 -> 4 ──────────────────────────────────────────────────────────────»
                                                                             »
       meas: 2/══════════════════════════════════════════════════════════════»
                                    

AttributeError: module 'qiskit.result' has no attribute 'get_counts'

In [117]:
for qc in circs:
    counts = results.get_counts(qc)
    print(qc)
    print(counts)

/Users/scubagibs/anaconda3/envs/QRESEARCH_ENV/lib/python3.9/site-packages/qiskit/providers/ibmq/managed/managedjobset.py:394: DeprecationWarning: The ``IBMQJob.qobj()`` method is deprecated and will be removed in a future release. You can now pass circuits to ``IBMQBackend.run()`` and use ``IBMQJob.circuits()``, ``IBMQJob.backend_options()``, and ``IBMQJob.header()`` to retrieve circuits, run configuration, and Qobj header, respectively.
  for i, exp in enumerate(job.qobj().experiments):
/Users/scubagibs/anaconda3/envs/QRESEARCH_ENV/lib/python3.9/site-packages/qiskit/providers/ibmq/managed/managedjobset.py:394: DeprecationWarning: The ``IBMQJob.qobj()`` method is deprecated and will be removed in a future release. You can now pass circuits to ``IBMQBackend.run()`` and use ``IBMQJob.circuits()``, ``IBMQJob.backend_options()``, and ``IBMQJob.header()`` to retrieve circuits, run configuration, and Qobj header, respectively.
  for i, exp in enumerate(job.qobj().experiments):


global phase: 0
                ░ ┌─────────┐┌───┐┌──────────┐┌───┐┌────┐┌────────────┐┌────┐»
      q_0 -> 0 ─░─┤ RZ(π/2) ├┤ X ├┤ RZ(3π/2) ├┤ X ├┤ √X ├┤ RZ(4.2487) ├┤ √X ├»
                ░ └─────────┘└─┬─┘└──────────┘└─┬─┘└────┘└────────────┘└────┘»
      q_1 -> 1 ─░──────────────■────────────────■────────────────────────────»
                ░                                                            »
ancilla_0 -> 2 ──────────────────────────────────────────────────────────────»
                                                                             »
ancilla_1 -> 3 ──────────────────────────────────────────────────────────────»
                                                                             »
ancilla_2 -> 4 ──────────────────────────────────────────────────────────────»
                                                                             »
       meas: 2/══════════════════════════════════════════════════════════════»
                                    

/Users/scubagibs/anaconda3/envs/QRESEARCH_ENV/lib/python3.9/site-packages/qiskit/providers/ibmq/managed/managedjobset.py:394: DeprecationWarning: The ``IBMQJob.qobj()`` method is deprecated and will be removed in a future release. You can now pass circuits to ``IBMQBackend.run()`` and use ``IBMQJob.circuits()``, ``IBMQJob.backend_options()``, and ``IBMQJob.header()`` to retrieve circuits, run configuration, and Qobj header, respectively.
  for i, exp in enumerate(job.qobj().experiments):
